In [1]:
# %% [markdown]
# # StarGAN v2 (CelebA-HQ) — Génération & Évaluation (Windows / CPU)
# Ce notebook télécharge le repo StarGAN v2, charge les checkpoints CelebA-HQ,
# génère des images 256x256, et calcule FID / Inception Score / LPIPS.

# %% 
import os, io, sys, zipfile, requests, glob, json, math, random, shutil, subprocess
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import numpy as np

# Limiter les threads pour éviter les conflits OpenMP sous Windows
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"  # en dernier recours si libiomp doublé

# Dossiers
ROOT      = Path(r"C:\Users\ilyes\Downloads\stylegan2_cond")      # racine projet
EXT       = ROOT / "ext"
EXT.mkdir(parents=True, exist_ok=True)
OUT_ROOT  = ROOT / "runs" / "eval_two_models"
OUT_ROOT.mkdir(parents=True, exist_ok=True)

# Chemin des images réelles (FairFace filtrées)
REAL_ROOT = ROOT / "FilteredFaces"   # -> mets ici ton dossier d'images réelles
assert REAL_ROOT.exists(), f"Le dossier réel {REAL_ROOT} n'existe pas."

# Réglages généraux
SEED        = 123
random.seed(SEED); np.random.seed(SEED)
IMG_SIZE    = 256
N_GEN       = 200   # commence petit sur CPU; augmente après validation
BATCH_GEN   = 8

import torch
DEVICE = torch.device("cpu")   # mets "cuda" si GPU
print("DEVICE:", DEVICE)

# Installations indispensables (en cellule séparée si besoin)
def pip_install(pkgs):
    cmd = [sys.executable, "-m", "pip", "install", "-q"] + pkgs
    print(">> pip install", " ".join(pkgs))
    subprocess.run(cmd, check=True)

pip_install(["munch", "torchmetrics[image]", "torch-fidelity", "lpips"])


DEVICE: cpu
>> pip install munch torchmetrics[image] torch-fidelity lpips


In [2]:
# %%
SGV2_DIR = EXT / "stargan-v2"

if not SGV2_DIR.exists():
    print("Téléchargement du repo StarGAN v2 (ZIP)...")
    url = "https://github.com/clovaai/stargan-v2/archive/refs/heads/master.zip"
    z = requests.get(url, timeout=60).content
    with zipfile.ZipFile(io.BytesIO(z)) as zf:
        zf.extractall(EXT)
    (EXT / "stargan-v2-master").rename(SGV2_DIR)
    print("OK:", SGV2_DIR)
else:
    print("Déjà présent:", SGV2_DIR)

# Rendre importables les modules du repo
if str(SGV2_DIR) not in sys.path:
    sys.path.insert(0, str(SGV2_DIR))

# Optionnel: certains forks utilisent 'core', d'autres 'core.networks' etc.
print("Contenu SGV2:", [p.name for p in SGV2_DIR.iterdir() if p.is_dir()])


Téléchargement du repo StarGAN v2 (ZIP)...
OK: C:\Users\ilyes\Downloads\stylegan2_cond\ext\stargan-v2
Contenu SGV2: ['assets', 'core', 'metrics']


In [4]:
# %%
import os, sys, io, requests, zipfile
from pathlib import Path

ROOT = Path(r"C:\Users\ilyes\Downloads\stylegan2_cond")
SGV2_DIR = ROOT / "ext" / "stargan-v2"
CKPT_DIR = SGV2_DIR / "expr" / "checkpoints" / "celeba_hq"
CKPT_DIR.mkdir(parents=True, exist_ok=True)

# Liens issus de download.sh (ClovaAI) — on force dl=1 pour un téléchargement direct.
URL_CELEBAHQ = "https://www.dropbox.com/s/96fmei6c93o8b8t/100000_nets_ema.ckpt?dl=1"

def download_file(url: str, dest: Path, chunk=1<<20):
    print(f"Téléchargement → {dest.name}")
    r = requests.get(url, stream=True, timeout=120)
    r.raise_for_status()
    total = int(r.headers.get("content-length", 0))
    done = 0
    with open(dest, "wb") as f:
        for c in r.iter_content(chunk_size=chunk):
            if c:
                f.write(c); done += len(c)
                if total:
                    pct = 100*done/total
                    print(f"\r  {done/1e6:.1f} MB / {total/1e6:.1f} MB ({pct:.1f}%)", end="")
    print("\nOK.")

ckpt_path = CKPT_DIR / "100000_nets_ema.ckpt"
if not ckpt_path.exists():
    download_file(URL_CELEBAHQ, ckpt_path)
else:
    print("Déjà présent:", ckpt_path)

print("Chemin checkpoints:", CKPT_DIR, "| Contenu:", [p.name for p in CKPT_DIR.iterdir()])


Téléchargement → 100000_nets_ema.ckpt
  292.8 MB / 292.8 MB (100.0%)
OK.
Chemin checkpoints: C:\Users\ilyes\Downloads\stylegan2_cond\ext\stargan-v2\expr\checkpoints\celeba_hq | Contenu: ['100000_nets_ema.ckpt']


In [6]:
# %% [markdown]
# StarGAN v2 (CelebA-HQ) — génération + évaluation (Windows, sans bash)

# %%
import os, sys, io, requests, zipfile, glob, random
from pathlib import Path

# --- Chemins à ADAPTER si besoin ---
ROOT = Path(r"C:\Users\ilyes\Downloads\stylegan2_cond")
REAL_ROOT = ROOT / "FilteredFaces"                  # dossier avec tes images FairFace filtrées
EXT = ROOT / "ext"
EXT.mkdir(parents=True, exist_ok=True)

# Repo StarGAN v2 (si pas déjà extrait)
SGV2_DIR = EXT / "stargan-v2"
if not SGV2_DIR.exists():
    print("Téléchargement du repo StarGAN v2 (ZIP)…")
    url = "https://github.com/clovaai/stargan-v2/archive/refs/heads/master.zip"
    z = requests.get(url, timeout=120).content
    with zipfile.ZipFile(io.BytesIO(z)) as zf:
        zf.extractall(EXT)
    (EXT / "stargan-v2-master").rename(SGV2_DIR)
    print("OK:", SGV2_DIR)
else:
    print("Repo présent:", SGV2_DIR)

# Dossier checkpoints & ckpt CelebA-HQ
CKPT_DIR = SGV2_DIR / "expr" / "checkpoints" / "celeba_hq"
CKPT_DIR.mkdir(parents=True, exist_ok=True)
CKPT_FILE = CKPT_DIR / "100000_nets_ema.ckpt"

# Télécharge le ckpt officiel (équivalent au download.sh)
if not CKPT_FILE.exists():
    print("Téléchargement du checkpoint CelebA-HQ…")
    url_ckpt = "https://www.dropbox.com/s/96fmei6c93o8b8t/100000_nets_ema.ckpt?dl=1"
    r = requests.get(url_ckpt, stream=True, timeout=300); r.raise_for_status()
    with open(CKPT_FILE, "wb") as f:
        for chunk in r.iter_content(chunk_size=1<<20):
            if chunk: f.write(chunk)
    print("OK:", CKPT_FILE)
else:
    print("Checkpoint présent:", CKPT_FILE)

# (Optionnel) wing.ckpt si ta révision le demande (sinon on mettra w_hpf=0 et on n'en a pas besoin)
# WING_DIR = SGV2_DIR / "expr" / "checkpoints"
# WING_DIR.mkdir(parents=True, exist_ok=True)
# for url, name in [
#     ("https://www.dropbox.com/s/tjxpypwpt38926e/wing.ckpt?dl=1", "wing.ckpt"),
#     ("https://www.dropbox.com/s/91fth49gyb7xksk/celeba_lm_mean.npz?dl=1", "celeba_lm_mean.npz")
# ]:
#     p = WING_DIR / name
#     if not p.exists():
#         print("Téléchargement", name)
#         rr = requests.get(url, timeout=120); rr.raise_for_status()
#         p.write_bytes(rr.content)

# Dépendances métriques
import subprocess
def pip_install(pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", *pkgs])

pip_install(["torchmetrics>=1.2.0", "lpips", "pillow", "tqdm"])

import torch
print("PyTorch:", torch.__version__, "| Device:", "cuda" if torch.cuda.is_available() else "cpu")


Repo présent: C:\Users\ilyes\Downloads\stylegan2_cond\ext\stargan-v2
Checkpoint présent: C:\Users\ilyes\Downloads\stylegan2_cond\ext\stargan-v2\expr\checkpoints\celeba_hq\100000_nets_ema.ckpt
PyTorch: 2.5.1 | Device: cpu


In [11]:
# ==== StarGAN v2 (CelebA-HQ) — chargement robuste du ckpt et génération de test (FIX) ====
import os, sys, inspect, torch, random
from pathlib import Path
import numpy as np
from PIL import Image
from torchvision import transforms as T

# --- chemins à ADAPTER si nécessaire ---
SGV2_DIR  = Path(r"C:\Users\ilyes\Downloads\stylegan2_cond\ext\stargan-v2")
CKPT_FILE = SGV2_DIR / "expr" / "checkpoints" / "celeba_hq" / "100000_nets_ema.ckpt"
REAL_ROOT = Path(r"C:\Users\ilyes\Downloads\stylegan2_cond\FilteredFaces")  # 1-2 images pour test
OUT_DIR   = Path(r"C:\Users\ilyes\Downloads\stylegan2_cond\runs\eval_two_models\starganv2_celebahq_fix")
OUT_DIR.mkdir(parents=True, exist_ok=True)

assert CKPT_FILE.exists(), f"Checkpoint introuvable: {CKPT_FILE}"
assert SGV2_DIR.exists(), "Repo StarGAN v2 introuvable"
sys.path.insert(0, str(SGV2_DIR))

# Import compatible selon la révision du repo
try:
    from core.model import Generator, MappingNetwork, StyleEncoder  # révision A
except Exception:
    from core.networks import Generator, MappingNetwork, StyleEncoder  # révision B

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_grad_enabled(False)

def strip_module_prefix(sd: dict):
    out = {}
    for k, v in sd.items():
        # ✅ FIX: construire la clé, puis affecter
        key = k[len("module."):] if k.startswith("module.") else k
        out[key] = v
    return out

def first_present(d, keys, default=None):
    for k in keys:
        if k in d: 
            return d[k]
    return default

# 1) Charger ckpt
ckpt = torch.load(str(CKPT_FILE), map_location="cpu")
root = ckpt.get("nets_ema", ckpt.get("state_dict", ckpt))
args = ckpt.get("args", {}) if isinstance(ckpt.get("args", {}), dict) else {}

# 2) Lire/hardcoder la config (valeurs CelebA-HQ officielles par défaut)
img_size     = int(args.get("img_size", 256))
style_dim    = int(args.get("style_dim", 64))
latent_dim   = int(args.get("latent_dim", 16))
num_domains  = int(args.get("num_domains", 2))
w_hpf        = int(args.get("w_hpf", 0))
max_conv_dim = int(args.get("max_conv_dim", 512))  # ⭐ critique (512 dans le checkpoint CelebA-HQ)

print(f"Config cible: img_size={img_size} style_dim={style_dim} latent_dim={latent_dim} "
      f"num_domains={num_domains} w_hpf={w_hpf} max_conv_dim={max_conv_dim}")

# 3) Fabrique un Generator en respectant la signature réelle (max_conv_dim ou F0)
def build_generator_try(img_size, style_dim, w_hpf, max_conv_dim):
    params = inspect.signature(Generator).parameters
    kwargs = {"w_hpf": w_hpf}

    if "max_conv_dim" in params:
        kwargs["max_conv_dim"] = max_conv_dim
        print("→ Generator(..., max_conv_dim=", max_conv_dim, ")")
        return Generator(img_size, style_dim, **kwargs)

    if "F0" in params:
        # Dans certains forks, F0=64 ~ 512 canaux max
        for F0 in (64, 96, 128):
            try:
                print(f"→ Generator(..., F0={F0})")
                return Generator(img_size, style_dim, F0=F0, **kwargs)
            except TypeError:
                continue
        print("→ Generator(...) sans F0/max_conv_dim (fallback)")
        return Generator(img_size, style_dim, **kwargs)

    print("→ Generator(...) (signature non standard)")
    return Generator(img_size, style_dim, **kwargs)

# 4) Récupérer state_dicts
g_sd = first_present(root, ["generator_ema","generator","G_ema","G"])
m_sd = first_present(root, ["mapping_network_ema","mapping_network","M_ema","M"])
assert g_sd is not None and m_sd is not None, "Clés du générateur ou mapping absentes dans le ckpt."

g_sd = strip_module_prefix(g_sd)
m_sd = strip_module_prefix(m_sd)

# 5) Essayer différentes variantes jusqu'à ce que le load_state_dict passe
variants = [
    dict(img_size=img_size, style_dim=style_dim, w_hpf=w_hpf, max_conv_dim=max_conv_dim),  # 512 espéré
    dict(img_size=img_size, style_dim=style_dim, w_hpf=w_hpf, max_conv_dim=1024),
    dict(img_size=img_size, style_dim=style_dim, w_hpf=w_hpf, max_conv_dim=576),
]

last_err = None
G = None
for v in variants:
    try:
        G_try = build_generator_try(**v).to(DEVICE).eval()
        # strict=False pour tolérer quelques buffers non-critiques
        missing_g, unexpected_g = G_try.load_state_dict(g_sd, strict=False)
        print(f"G chargé | missing={len(missing_g)} unexpected={len(unexpected_g)}")
        G = G_try
        break
    except Exception as e:
        print("Echec avec variante:", v, "\n  ↳", repr(e))
        last_err = e
        continue

if G is None:
    raise RuntimeError("Impossible d'instancier le Generator compatible avec le ckpt.") from last_err

# MappingNetwork
M = MappingNetwork(latent_dim, style_dim, num_domains).to(DEVICE).eval()
missing_m, unexpected_m = M.load_state_dict(m_sd, strict=False)
print(f"M chargé | missing={len(missing_m)} unexpected={len(unexpected_m)}")

# 6) Test rapide : générer 2 sorties à partir de 2 vraies images
paths = [p for p in REAL_ROOT.rglob("*") if p.suffix.lower() in {".jpg",".jpeg",".png"}]
if len(paths) < 2:
    raise FileNotFoundError(f"Il faut au moins 2 images dans {REAL_ROOT} pour le test.")

def load_real_img(p, size=img_size):
    tfm = T.Compose([T.Resize(size), T.CenterCrop(size), T.ToTensor()])
    return tfm(Image.open(p).convert("RGB")).unsqueeze(0)*2 - 1  # [-1,1]

x_src = torch.cat([load_real_img(paths[0]), load_real_img(paths[1])], dim=0).to(DEVICE)
y_trg = torch.randint(0, num_domains, (x_src.size(0),), device=DEVICE)
z_trg = torch.randn(x_src.size(0), latent_dim, device=DEVICE)
with torch.no_grad():
    s_trg = M(z_trg, y_trg)
    x_fake = G(x_src, s_trg)                      # [-1,1]
x_fake = ((x_fake.clamp_(-1,1)+1)/2).cpu().numpy()  # [0,1]

for i in range(x_fake.shape[0]):
    arr = (x_fake[i].transpose(1,2,0)*255).astype(np.uint8)
    Image.fromarray(arr).save(OUT_DIR / f"demo_{i:02d}.png")
print("Images écrites dans:", OUT_DIR)


C:\Users\ilyes\AppData\Local\Temp\ipykernel_25964\574996640.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(str(CKPT_FILE), map_location="cpu")


Config cible: img_size=256 style_dim=64 latent_dim=16 num_domains=2 w_hpf=0 max_conv_dim=512
→ Generator(..., max_conv_dim= 512 )
Echec avec variante: {'img_size': 256, 'style_dim': 64, 'w_hpf': 0, 'max_conv_dim': 512} 
  ↳ RuntimeError('Error(s) in loading state_dict for Generator:\n\tsize mismatch for decode.3.conv1.weight: copying a param with shape torch.Size([512, 512, 3, 3]) from checkpoint, the shape in current model is torch.Size([256, 512, 3, 3]).\n\tsize mismatch for decode.3.conv1.bias: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([256]).\n\tsize mismatch for decode.3.conv2.weight: copying a param with shape torch.Size([512, 512, 3, 3]) from checkpoint, the shape in current model is torch.Size([256, 256, 3, 3]).\n\tsize mismatch for decode.3.conv2.bias: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([256]).\n\tsize mismatch for decode.3.norm2.fc.weight: copying a

RuntimeError: Impossible d'instancier le Generator compatible avec le ckpt.